In [5]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# --- 1. Define Sets ---
# These sets represent the indices in your model.
# Updated based on your request: 2 products, 3 mfg plants, 4 dcs, 3 stores.
I_SET = range(1, 3)  # Designs/Items/Products (2 products: 1, 2)
J_SET = range(1, 4)  # Manufacturers/Mfg Plants (3 manufacturers: 1, 2, 3)
K_SET = range(1, 5)  # Distribution Centers (4 DCs: 1, 2, 3, 4)
E_SET = range(1, 4)  # Stores (3 Stores: 1, 2, 3)

print(f"Sets defined: I={list(I_SET)}, J={list(J_SET)}, K={list(K_SET)}, E={list(E_SET)}")

# --- Mappings for string names to numerical IDs (for CSVs with names) ---
# These mappings are crucial for linking your CSV data to the numerical indices in the model.
# Adjust these if your CSVs use different names or if your sets have different sizes/names.

# Manufacturers (J_SET: 1, 2, 3)
# Based on 'cost_mfg_product.csv' and consistent with distance_in_km_manu_to_dc.csv columns
mfg_name_to_id = {'CAM': 1, 'LEV': 2, 'LETH': 3}
id_to_mfg_name = {v: k for k, v in mfg_name_to_id.items()}

# Products/Items (I_SET: 1, 2)
# Based on 'cost_mfg_product.csv' and 'store_demand_data.csv'
product_name_to_id = {'Product 1': 1, 'Product 2': 2}
id_to_product_name = {v: k for k, v in product_name_to_id.items()}

# Stores (E_SET: 1, 2, 3)
# Based on 'store_demand_data.csv' and 'fixedcost_dc_to_store.csv' rows
store_name_to_id = {'MONT': 1, 'NEW': 2, 'KEL': 3}
id_to_store_name = {v: k for k, v in store_name_to_id.items()}

# Distribution Centers (K_SET: 1, 2, 3, 4)
# Based on 'fixedcost_dc_to_store.csv' columns and 'distance_in_km_manu_to_dc.csv' rows
dc_name_to_id = {'CORN': 1, 'SUR': 2, 'CAL': 3, 'VAU': 4}
id_to_dc_name = {v: k for k, v in dc_name_to_id.items()}


print("\nMappings:")
print(f"  Manufacturers: {mfg_name_to_id}")
print(f"  Products: {product_name_to_id}")
print(f"  Stores: {store_name_to_id}")
print(f"  Distribution Centers: {dc_name_to_id}")


# --- 2. Load Data ---

# Load c_ij (cost of producing item i by manufacturer j)
# From 'cost_mfg_product.csv' (MFG, Product 1, Product 2)
df_c_ij = pd.read_csv('cost_mfg_product.csv')
c_ij = {}
for index, row in df_c_ij.iterrows():
    mfg_id = mfg_name_to_id.get(row['MFG'])
    # Removed: if mfg_id is None: raise ValueError(...)
    for col in ['Product 1', 'Product 2']: # Explicitly iterate over product columns
        product_id = product_name_to_id.get(col)
        # Removed: if product_id is None: raise ValueError(...)
        if mfg_id is not None and product_id is not None: # Only add if both IDs are valid
            c_ij[(product_id, mfg_id)] = row[col]
print("\nProduction Costs (c_ij):", c_ij)


# Load ctf_jk (fixed cost of sending item i from manufacturer j to distribution center k)
# From 'fixedcost_manu_to_dc.csv' (Rows: DCs, Cols: Manufacturers) -> Transposed to (Mfg, DC)
df_ctf_jk = pd.read_csv('fixedcost_manu_to_dc.csv')
ctf_jk = {}
# Assuming the first column is unnamed and contains DC names
df_ctf_jk.rename(columns={df_ctf_jk.columns[0]: 'DC_Name'}, inplace=True)
for index, row in df_ctf_jk.iterrows():
    dc_id = dc_name_to_id.get(row['DC_Name'])
    # Removed: if dc_id is None: raise ValueError(...)

    for col_name in df_ctf_jk.columns[1:]: # Iterate over Manufacturer columns
        mfg_id = mfg_name_to_id.get(col_name)
        # Removed: if mfg_id is None: raise ValueError(...)
        # Store as (Manufacturer, DC)
        if mfg_id is not None and dc_id is not None: # Only add if both IDs are valid
            ctf_jk[(mfg_id, dc_id)] = row[col_name]
print("\nManufacturer-DC Fixed Costs (ctf_jk):", ctf_jk)


# Load ctv_ijk (variable cost of sending item i from manufacturer j to distribution center k)
# From 'variablecost_manu_to_dc_prod1.csv' and 'variablecost_manu_to_dc_prod2.csv'
# (Rows: DCs, Cols: Manufacturers) -> Transposed to (Product, Mfg, DC)
ctv_ijk = {}
for prod_id, prod_file in [(1, 'variablecost_manu_to_dc_prod1.csv'), (2, 'variablecost_manu_to_dc_prod2.csv')]:
    df_ctv_ijk = pd.read_csv(prod_file)
    df_ctv_ijk.rename(columns={df_ctv_ijk.columns[0]: 'DC_Name'}, inplace=True)
    for index, row in df_ctv_ijk.iterrows():
        dc_id = dc_name_to_id.get(row['DC_Name'])
        # Removed: if dc_id is None: raise ValueError(...)

        for col_name in df_ctv_ijk.columns[1:]: # Iterate over Manufacturer columns
            mfg_id = mfg_name_to_id.get(col_name)
            # Removed: if mfg_id is None: raise ValueError(...)
            # Store as (Product, Manufacturer, DC)
            if dc_id is not None and mfg_id is not None: # Only add if both IDs are valid
                ctv_ijk[(prod_id, mfg_id, dc_id)] = row[col_name]
print("\nManufacturer-DC Variable Costs (ctv_ijk):", ctv_ijk)


# Load dist_jk (distance from manufacturer j to distribution center k)
# From 'distance_in_km_manu_to_dc.csv' (Rows: DCs, Cols: Manufacturers) -> Transposed to (Mfg, DC)
df_dist_jk = pd.read_csv('distance_in_km_manu_to_dc.csv')
dist_jk = {}
df_dist_jk.rename(columns={df_dist_jk.columns[0]: 'DC_Name'}, inplace=True)
for index, row in df_dist_jk.iterrows():
    dc_id = dc_name_to_id.get(row['DC_Name'])
    # Removed: if dc_id is None: raise ValueError(...)
    for col_name in df_dist_jk.columns[1:]: # Iterate over Manufacturer columns
        mfg_id = mfg_name_to_id.get(col_name)
        # Removed: if mfg_id is None: raise ValueError(...)
        # Store as (Manufacturer, DC)
        if dc_id is not None and mfg_id is not None: # Only add if both IDs are valid
            dist_jk[(mfg_id, dc_id)] = row[col_name]
print("\nDistances Manu to DC (dist_jk):", dist_jk)


# Load ctf_ke (fixed cost of sending item i from distribution center k to store e)
# From 'fixedcost_dc_to_store.csv' (Rows: Stores, Cols: DCs) -> Transposed to (DC, Store)
df_ctf_ke = pd.read_csv('fixedcost_dc_to_store.csv')
df_ctf_ke.rename(columns={df_ctf_ke.columns[0]: 'Store_Name'}, inplace=True)
ctf_ke = {}
for index, row in df_ctf_ke.iterrows():
    store_id = store_name_to_id.get(row['Store_Name'])
    # Removed: if store_id is None: raise ValueError(...)
    for col_name in df_ctf_ke.columns[1:]: # Iterate over DC columns
        dc_id = dc_name_to_id.get(col_name)
        # Removed: if dc_id is None: raise ValueError(...)
        # Store as (DC, Store)
        if store_id is not None and dc_id is not None: # Only add if both IDs are valid
            ctf_ke[(dc_id, store_id)] = row[col_name]
print("\nDC-Store Fixed Costs (ctf_ke):", ctf_ke)


# Load ctv_ike (variable cost of sending item i from distribution center k to store e)
# From 'variablecost_dc_to_store_prod1.csv' and 'variablecost_dc_to_store_prod2.csv'
# (Rows: Stores, Cols: DCs) -> Transposed to (Product, DC, Store)
ctv_ike = {}
for prod_id, prod_file in [(1, 'variablecost_dc_to_store_prod1.csv'), (2, 'variablecost_dc_to_store_prod2.csv')]:
    df_ctv_ike = pd.read_csv(prod_file)
    df_ctv_ike.rename(columns={df_ctv_ike.columns[0]: 'Store_Name'}, inplace=True)
    for index, row in df_ctv_ike.iterrows():
        store_id = store_name_to_id.get(row['Store_Name'])
        # Removed: if store_id is None: raise ValueError(...)
        for col_name in df_ctv_ike.columns[1:]: # Iterate over DC columns
            dc_id = dc_name_to_id.get(col_name)
            # Removed: if dc_id is None: raise ValueError(...)
            # Store as (Product, DC, Store)
            if store_id is not None and dc_id is not None: # Only add if both IDs are valid
                ctv_ike[(prod_id, dc_id, store_id)] = row[col_name]
print("\nDC-Store Variable Costs (ctv_ike):", ctv_ike)


# Load dist_ke (distance from distribution center k to store e)
# From 'distance_in_km_dc_to_store.csv' (Rows: Stores, Cols: DCs) -> Transposed to (DC, Store)
df_dist_ke = pd.read_csv('distance_in_km_dc_to_store.csv')
dist_ke = {}
df_dist_ke.rename(columns={df_dist_ke.columns[0]: 'Store_Name'}, inplace=True)
for index, row in df_dist_ke.iterrows():
    store_id = store_name_to_id.get(row['Store_Name'])
    # Removed: if store_id is None: raise ValueError(...)
    for col_name in df_dist_ke.columns[1:]: # Iterate over DC columns
        dc_id = dc_name_to_id.get(col_name)
        # Removed: if dc_id is None: raise ValueError(...)
        # Store as (DC, Store)
        if store_id is not None and dc_id is not None: # Only add if both IDs are valid
            dist_ke[(dc_id, store_id)] = row[col_name]
print("\nDistances DC to Store (dist_ke):", dist_ke)


# Load cap_j (capacity of manufacturer j)
# Provided directly in query
cap_j = {
    mfg_name_to_id['CAM']: 151000,
    mfg_name_to_id['LEV']: 91000,
    mfg_name_to_id['LETH']: 61000,
}
print("\nManufacturer Capacities (cap_j):", cap_j)


# Load cap_k (capacity of distribution center k)
# Provided directly in query
cap_k = {
    dc_name_to_id['CORN']: 111000,
    dc_name_to_id['SUR']: 95000,
    dc_name_to_id['CAL']: 63000,
    dc_name_to_id['VAU']: 48000,
}
print("\nDC Capacities (cap_k):", cap_k)


# Load cap_e (capacity of store e)
# Provided directly in query
cap_e = {
    store_name_to_id['MONT']: 75000,
    store_name_to_id['NEW']: 77000,
    store_name_to_id['KEL']: 61000,
}
print("\nStore Capacities (cap_e):", cap_e)


# Load dem_ie (demand for item i in store e)
# Assuming 'store_demand_data.csv' has columns: 'Store_ID', 'Product 1 Demand', 'Product 2 Demand'
df_dem_ie = pd.read_csv('store_demand_data.csv')
dem_ie = {}
for index, row in df_dem_ie.iterrows():
    store_id = store_name_to_id.get(row['Store_ID'])
    # Removed: if store_id is None: raise ValueError(...)
    if store_id is not None: # Only process if store_id is valid
        dem_ie[(product_name_to_id['Product 1'], store_id)] = row['Product 1 Demand']
        dem_ie[(product_name_to_id['Product 2'], store_id)] = row['Product 2 Demand']
print("\nStore Demands (dem_ie):", dem_ie)


# --- 3. Create Gurobi Model ---
model = gp.Model("SupplyChainOptimization")

# --- 4. Define Decision Variables ---

# trans_ijk: number of item i to be sent from manufacturer j to distribution center k
trans_ijk = model.addVars(I_SET, J_SET, K_SET, name="trans_ijk", vtype=GRB.CONTINUOUS, lb=0.0)

# trans_ike: number of item i to be sent from distribution center k to store e
trans_ike = model.addVars(I_SET, K_SET, E_SET, name="trans_ike", vtype=GRB.CONTINUOUS, lb=0.0)

# x_ij: number of item i to be produced by manufacturer j
x_ij = model.addVars(I_SET, J_SET, name="x_ij", vtype=GRB.CONTINUOUS, lb=0.0)

# Binary variables for route activation
y_jk = model.addVars(J_SET, K_SET, name="y_jk", vtype=GRB.BINARY) # 1 if route from manu j to DC k is open
z_ke = model.addVars(K_SET, E_SET, name="z_ke", vtype=GRB.BINARY) # 1 if route from DC k to store e is open

# --- 5. Set Objective Function ---
# Min z = Sum(i,j) x_ij * c_ij
#         + Sum(j,k) ctf_jk * y_jk (Fixed cost for opening Manu-DC route)
#         + Sum(k,e) ctf_ke * z_ke (Fixed cost for opening DC-Store route)
#         + Sum(i,j,k) ctv_ijk * dist_jk * trans_ijk (Variable cost Manu-DC)
#         + Sum(i,k,e) ctv_ike * dist_ke * trans_ike (Variable cost DC-Store)

obj_production_cost = gp.quicksum(x_ij[i, j] * c_ij[i, j] for i in I_SET for j in J_SET)

obj_fixed_manu_dc_cost = gp.quicksum(ctf_jk[j, k] * y_jk[j, k] for j in J_SET for k in K_SET)

obj_fixed_dc_store_cost = gp.quicksum(ctf_ke[k, e] * z_ke[k, e] for k in K_SET for e in E_SET)

obj_variable_manu_dc_cost = gp.quicksum(ctv_ijk[i, j, k] * dist_jk[j, k] * trans_ijk[i, j, k]
                                        for i in I_SET for j in J_SET for k in K_SET)

obj_variable_dc_store_cost = gp.quicksum(ctv_ike[i, k, e] * dist_ke[k, e] * trans_ike[i, k, e]
                                         for i in I_SET for k in K_SET for e in E_SET)

model.setObjective(obj_production_cost + obj_fixed_manu_dc_cost + obj_fixed_dc_store_cost +
                   obj_variable_manu_dc_cost + obj_variable_dc_store_cost, GRB.MINIMIZE)

# --- 6. Add Constraints ---

# A large number M for linking constraints
M = 1000000000 # Increased M to be sufficiently large for new capacities/demands

# Constraint (2): Manufacturer capacity
# Sum(i) x_ij <= cap_j for all j
for j in J_SET:
    model.addConstr(gp.quicksum(x_ij[i, j] for i in I_SET) <= cap_j[j], f"ManufacturerCapacity_{j}")

# Constraint (3): Producer to DC transfer limit
# x_ij - Sum(k) trans_ijk >= 0 for all i, j
for i in I_SET:
    for j in J_SET:
        model.addConstr(x_ij[i, j] - gp.quicksum(trans_ijk[i, j, k] for k in K_SET) >= 0, f"ProducerToDCLimit_{i}_{j}")

# Constraint (4): DC capacity (inbound)
# Sum(i) Sum(j) trans_ijk <= cap_k for all k
for k in K_SET:
    model.addConstr(gp.quicksum(trans_ijk[i, j, k] for i in I_SET for j in J_SET) <= cap_k[k], f"DCCapacityInbound_{k}")

# Constraint (5): Flow balance at DC
# Sum(j) trans_ijk - Sum(e) trans_ike >= 0 for all i, k
# This ensures that what comes into a DC for a given item is at least what leaves it.
for i in I_SET:
    for k in K_SET:
        model.addConstr(gp.quicksum(trans_ijk[i, j, k] for j in J_SET) - gp.quicksum(trans_ike[i, k, e] for e in E_SET) >= 0, f"DCFlowBalance_{i}_{k}")

# Constraint (6): Store capacity (inbound)
# Sum(i) Sum(k) trans_ike <= cap_e for all e
for e in E_SET:
    model.addConstr(gp.quicksum(trans_ike[i, k, e] for i in I_SET for k in K_SET) <= cap_e[e], f"StoreCapacityInbound_{e}")

# Constraint (7): Store demand fulfillment
# Sum(k) trans_ike >= dem_ie for all i, e
for i in I_SET:
    for e in E_SET:
        model.addConstr(gp.quicksum(trans_ike[i, k, e] for k in K_SET) >= dem_ie[i, e], f"StoreDemandFulfillment_{i}_{e}")

# Linking Constraints: Connect continuous flow variables with binary route activation variables
# If a route is not open (y_jk = 0), no items can flow through it (trans_ijk = 0)
for i in I_SET:
    for j in J_SET:
        for k in K_SET:
            model.addConstr(trans_ijk[i, j, k] <= M * y_jk[j, k], f"Link_ManuDC_{i}_{j}_{k}")

for i in I_SET:
    for k in K_SET:
        for e in E_SET:
            model.addConstr(trans_ike[i, k, e] <= M * z_ke[k, e], f"Link_DCStore_{i}_{k}_{e}")

# --- 7. Optimize the Model ---
model.optimize()

# --- 8. Print Results ---
if model.status == GRB.OPTIMAL:
    print("\n--- Optimization Successful ---")
    print(f"Optimal Objective Value: {model.objVal:.2f}")

    print("\n--- Decision Variable Values ---")
    print("x_ij (Production by Manufacturer):")
    for i in I_SET:
        for j in J_SET:
            if x_ij[i, j].x > 1e-6: # Print only non-zero values
                print(f"  Item {id_to_product_name.get(i, i)} by Manufacturer {id_to_mfg_name.get(j, j)}: {x_ij[i, j].x:.2f}")

    print("\ny_jk (Manufacturer-DC Route Open):")
    for j in J_SET:
        for k in K_SET:
            if y_jk[j, k].x > 0.5: # Binary variable, check if close to 1
                print(f"  Route from Manu {id_to_mfg_name.get(j, j)} to DC {id_to_dc_name.get(k, k)} is OPEN")

    print("\ntrans_ijk (Transfer from Manufacturer to DC):")
    for i in I_SET:
        for j in J_SET:
            for k in K_SET:
                if trans_ijk[i, j, k].x > 1e-6:
                    print(f"  Item {id_to_product_name.get(i, i)} from Manu {id_to_mfg_name.get(j, j)} to DC {id_to_dc_name.get(k, k)}: {trans_ijk[i, j, k].x:.2f}")

    print("\nz_ke (DC-Store Route Open):")
    for k in K_SET:
        for e in E_SET:
            if z_ke[k, e].x > 0.5: # Binary variable, check if close to 1
                print(f"  Route from DC {id_to_dc_name.get(k, k)} to Store {id_to_store_name.get(e, e)} is OPEN")

    print("\ntrans_ike (Transfer from DC to Store):")
    for i in I_SET:
        for k in K_SET:
            for e in E_SET:
                if trans_ike[i, k, e].x > 1e-6:
                    print(f"  Item {id_to_product_name.get(i, i)} from DC {id_to_dc_name.get(k, k)} to Store {id_to_store_name.get(e, e)}: {trans_ike[i, k, e].x:.2f}")

elif model.status == GRB.INFEASIBLE:
    print("\n--- Model is Infeasible ---")
    print("No solution satisfies all constraints. Consider checking your data and constraints.")
    # To diagnose infeasibility, you can use:
    # model.computeIIS()
    # model.write("infeasible_model.ilp")
elif model.status == GRB.UNBOUNDED:
    print("\n--- Model is Unbounded ---")
    print("The objective function can be made arbitrarily good. Check for missing constraints or bounds.")
else:
    print(f"\n--- Optimization ended with status {model.status} ---")


Sets defined: I=[1, 2], J=[1, 2, 3], K=[1, 2, 3, 4], E=[1, 2, 3]

Mappings:
  Manufacturers: {'CAM': 1, 'LEV': 2, 'LETH': 3}
  Products: {'Product 1': 1, 'Product 2': 2}
  Stores: {'MONT': 1, 'NEW': 2, 'KEL': 3}
  Distribution Centers: {'CORN': 1, 'SUR': 2, 'CAL': 3, 'VAU': 4}

Production Costs (c_ij): {(1, 1): 2.3, (2, 1): 3.6, (1, 2): 1.4, (2, 2): 3.1, (1, 3): 1.7, (2, 3): 3.3}

Manufacturer-DC Fixed Costs (ctf_jk): {(1, 1): 297.4944, (2, 1): 210.2592, (3, 1): 1848.7152, (1, 2): 2278.74, (2, 2): 2648.3712, (3, 2): 639.1656, (1, 3): 1766.5128, (2, 3): 2160.7488, (3, 3): 131.412, (1, 4): 52.78848, (2, 4): 463.0176, (3, 4): 1760.3616}

Manufacturer-DC Variable Costs (ctv_ijk): {(1, 1, 1): 0.16, (1, 2, 1): 0.21, (1, 3, 1): 0.3, (1, 1, 2): 0.2, (1, 2, 2): 0.2, (1, 3, 2): 0.27, (1, 1, 3): 0.32, (1, 2, 3): 0.35, (1, 3, 3): 0.37, (1, 1, 4): 0.21, (1, 2, 4): 0.28, (1, 3, 4): 0.36, (2, 1, 1): 0.39, (2, 2, 1): 0.45, (2, 3, 1): 0.59, (2, 1, 2): 0.4, (2, 2, 2): 0.55, (2, 3, 2): 0.6, (2, 1, 3): 0.